In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install QuantLib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 65.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

import QuantLib as ql
from scipy.optimize import minimize, differential_evolution, NonlinearConstraint

import os
os.chdir('/content/drive/MyDrive/Practicum')
from share_funcs import *

import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

# Model Calibration

Calibrating the Heston model is equivalent to solving the non-linear constrained optimization problem

$$
\begin{aligned}
& \min _{\Theta_{\text {Heston }} \in \mathbb{R}^5} F\left(\Theta_{\text {Heston }}\right), \\
& \text { s.t. } l \leq \Theta_{\text {Heston }} \leq u, \\
& \quad f_{\text {Feller }}\left(\Theta_{\text {Heston }}\right)<0,
\end{aligned}
$$

where the optimization objective function or error measure $F(\Theta_{Heston})$ defines the distance between the value of the instruments as computed by the Heston model for the parameter set $\Theta_{Heston} = \{v_0, k, 0, o, p\}$ and the corresponding market values or, as is the case in this paper, generated values. The optimization problem is subject to constraints on the parameters given by the lower (respectively upper) boundaries $l = (0, 0, 0, 0, -1)'$ and $u=(+\infty, +\infty, +\infty, +\infty, 1)'$ as well as the Feller condition $f_{Feller}(\Theta_{Heston})= \sigma^2- 2\kappa \theta < 0$.

## Procedure

1. Define the objective function
  $$
  f(x) = \sum^N_{i=1} (C_{market} - C_{Heston}(x))^2
  $$
  where
  - $C_{market}$ is the observed market price of the option
  - $C_{Heston}(x)$ is the price of the option predicted by the Heston model using parameter values x
  - x is a vector of Heston model parameters

2. Set an initial guess for the Heston parameters

3. Use an optimization algorithm to find the optimal set of parameters

  We can use an optimization algorithm, such as the **Levenberg-Marquardt algorithm** or **the Nelder-Mead simplex algorithm**, to find the set of Heston parameters that minimize the objective function. This involves iteratively adjusting the parameter values until the objective function is minimized.

4. Evaluate the quality of the calibration
  
  Once the optimal set of parameters is found, we need to evaluate the quality of the calibration. This can be done by comparing the predicted option prices to the observed market prices, calculating statistical measures such as the root-mean-square error or the correlation coefficient, and examining the residuals between the predicted and observed prices.

5. Refine the calibration if necessary

  If the calibration is not satisfactory, we can refine it by adjusting the initial guess for the parameters or by changing the optimization algorithm or its parameters. We may also need to consider using different types of options or different maturities in the calibration process.

## European Call Option - Fourier Transform Method

In [ ]:
# Generate the price of a European call option using the Fourier Transform Method with the Heston model
def eurocall_fourier(m, T, r, q, v0, theta, kappa, sigma, rho):
  '''
  m: Moneyness
  T: Time to maturity of the option
  r: Risk-free interest rate
  q: Dividend yield of the underlying asset
  v0: Initial volatility of the Heston mode
  theta: Mean reversion speed of the Heston model
  kappa: Mean reversion level of the Heston model
  sigma: Volatility of the Heston model
  rho: Correlation between the asset price and volatility processes
  '''

  # Parameters
  S0 = 1    # Current price of the underlying asset (scale)
  K = S0 * m    # Strike price of the option
  option_type = ql.Option.Call

  # Set up the option
  today = ql.Date.todaysDate()
  # expiry_date = today + ql.Period(T, ql.Days)
  expiry_date = today + ql.Period(f"{int(T*365)}d")
  option = ql.VanillaOption(ql.PlainVanillaPayoff(option_type, K), ql.EuropeanExercise(expiry_date))

  # Set up the Heston model
  heston_process = ql.HestonProcess(
      ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), r, ql.Actual365Fixed())),
      ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), q, ql.Actual365Fixed())),
      ql.QuoteHandle(ql.SimpleQuote(S0)),
      v0, kappa, theta, sigma, rho
      )
  heston_model = ql.HestonModel(heston_process)

  # Calculate the option price using the Heston model with the Fourier transform
  heston_engine = ql.AnalyticHestonEngine(heston_model, 64)
  option.setPricingEngine(heston_engine)
  # print("The price of the European call option is:", option.NPV())

  return option.NPV()

# Test
eurocall_fourier(1.05, 2, 0.05, 0.1, 0.05, 0.25, 1.0, 0.5, -0.5)

0.12374587649461455

## Data Generation

In [ ]:
def parameter_comb_generator(v0, theta, kappa, sigma, rho, no_of_comb):
  np.random.seed(124)

  paths = int(1.5 * no_of_comb)

  g_m = np.random.uniform(0.2, 2.5, paths)
  g_T = np.random.uniform(0.004, 2, paths)
  g_r = np.random.uniform(0., 0.08, paths)
  g_q = np.random.uniform(0., 0.05, paths)

  para_comb = pd.DataFrame({
      'm':g_m,
      'T':g_T,
      'r':g_r,
      'q':g_q,
      'v0':v0,
      'theta':theta,
      'kappa':kappa,
      'sigma':sigma,
      'rho':rho
      })

  # Feller condition
  para_comb.drop(para_comb[~(para_comb.sigma**2 < 2 * para_comb.kappa * para_comb.theta)].index, inplace=True)
  para_comb = para_comb[:no_of_comb]
  para_comb.reset_index(inplace=True)
  return para_comb

# para_comb = parameter_comb_generator(m, T, r, q, v0, theta, kappa, sigma, rho, 100000)
# para_comb.shape

def eurocall_fourier_generator(v0, theta, kappa, sigma, rho,
                               no_of_comb, file_name, error=1e-5):

  para_comb = parameter_comb_generator(v0, theta, kappa, sigma, rho, no_of_comb)

  g_eurocall = np.zeros(no_of_comb)
  for i in range(no_of_comb):
    call_price = eurocall_fourier(para_comb['m'][i], para_comb['T'][i], para_comb['r'][i], para_comb['q'][i],
                                  para_comb['v0'][i], para_comb['theta'][i], para_comb['kappa'][i], para_comb['sigma'][i], para_comb['rho'][i])
    err = call_price - .0
    g_eurocall[i] = call_price if err > error else .0

  file_path = "/content/drive/MyDrive/Practicum/data/" + file_name

  generated_data = pd.concat([para_comb, pd.DataFrame({'eurocall_ft': g_eurocall})], axis=1)
  generated_data.drop(['index'], axis=1, inplace=True)
  generated_data.to_csv(file_path)

  return generated_data

In [ ]:
# Heston Parameters
v0 = 0.04
theta = 0.04
kappa = 1.5
sigma = 0.3
rho = -0.5
actual_params = [v0, theta, kappa, sigma, rho]

# data for pricing model
# no_of_comb = 20000
# e_ft_pricing = eurocall_fourier_generator(v0, theta, kappa, sigma, rho, no_of_comb, 'eft_pricing_20k.csv')
# e_ft_pricing.head()

In [ ]:
# print('Percentage of 0:', e_ft_pricing[(e_ft_pricing.eurocall_ft == 0)].shape[0]/no_of_comb)
# print('Percentage of 0 when m >= 2:', e_ft_pricing[(e_ft_pricing.eurocall_ft == 0) & (e_ft_pricing.m >= 2)].shape[0]/no_of_comb)

Percentage of 0: 0.2558
Percentage of 0 when m >= 2: 0.1584


In [ ]:
# data for calibration
# no_of_comb = 20000
# e_fourier_test = eurocall_fourier_generator(v0, theta, kappa, sigma, rho, no_of_comb, 'eurocall_fourier_mc_20k2.csv')
# e_fourier_test.head()
e_ft = pd.read_csv('/content/drive/MyDrive/Practicum/data/eurocall_fourier_mc_20k2.csv', index_col=0)
e_ft.head()

,m,T,r,q,v0,theta,kappa,sigma,rho,eurocall_ft
0,0.443949,0.884297,0.019012,0.036236,0.04,0.04,1.5,0.3,-0.5,0.532060
1,1.914584,0.390791,0.041493,0.039068,0.04,0.04,1.5,0.3,-0.5,0.000000
2,1.516321,0.512961,0.053467,0.037099,0.04,0.04,1.5,0.3,-0.5,0.000027
3,1.253955,0.679612,0.052275,0.045176,0.04,0.04,1.5,0.3,-0.5,0.003914
4,1.084824,1.239035,0.079352,0.016475,0.04,0.04,1.5,0.3,-0.5,0.080850


In [ ]:
# data for LightGBM
no_of_comb = 5000
e_fourier_test = eurocall_fourier_generator(v0, theta, kappa, sigma, rho, no_of_comb, 'eurocall_fourier_test.csv')
e_fourier_test.head()

,m,T,r,q,v0,theta,kappa,sigma,rho,eurocall_ft
0,0.443949,1.292443,0.039235,0.027003,0.04,0.04,1.5,0.3,-0.5,0.544013
1,1.914584,1.441499,0.000564,0.018744,0.04,0.04,1.5,0.3,-0.5,0.000051
2,1.516321,0.889979,0.036836,0.014260,0.04,0.04,1.5,0.3,-0.5,0.000461
3,1.253955,0.009240,0.055564,0.032073,0.04,0.04,1.5,0.3,-0.5,0.000000
4,1.084824,0.029721,0.057135,0.014492,0.04,0.04,1.5,0.3,-0.5,0.000047


## Constrained Calibration

### Traditional Approach

In [ ]:
# params = v0, theta, kappa, sigma, rho
def Feller(params):
  '''
  params: a list of Heston model parameters [v0, theta, kappa, sigma, rho]
  The result should be negative.
  '''
  _, theta, kappa, sigma, _ = params
  return sigma**2 - 2*kappa*theta

def obj_fun(params, m, T, r, q, market_price):
  '''
  Calculates the residuals between observed and predicted option prices
  '''
  v0, theta, kappa, sigma, rho = params
  res = np.zeros_like(market_price)
  for i in range(len(market_price)):
    res[i] = eurocall_fourier(m[i], T[i], r[i], q[i], v0, theta, kappa, sigma, rho) - market_price[i]
  return np.sum(res**2)

def report_calibration(initial, actual, calibrated):
  initial1 = initial + [Feller(initial)]
  actual1 = actual + [Feller(actual)]
  calibrated1 = np.append(calibrated, Feller(calibrated))
  report = pd.DataFrame({"Initial": initial1, "Actual": actual1, "Calibrated": calibrated1},
                        index=['v0', 'theta', 'kappa', 'sigma', 'rho', 'Feller']).round(5).T
  return report

In [ ]:
# Initial Parameters
v00 = 0.1
theta0 = 0.05
kappa0 = 0.5
sigma0 = 0.2
rho0 = -0.7
params0 = [v00, theta0, kappa0, sigma0, rho0]

In [ ]:
bounds = ((1e-15, 0.5), (1e-15, 2), (1e-15, 2), (1e-15, 1), (-1, 1))
cons = {'type':'ineq', 'fun':Feller}
params_cal = minimize(obj_fun, x0=params0,
                      args=(e_ft['m'], e_ft['T'], e_ft['r'], e_ft['q'], e_ft['eurocall_ft']),
                      method='SLSQP', bounds=bounds,
                      constraints=cons, tol=1e-5, options={'maxiter':200}).x

print('Calibrated parameters: v0 = %.4f, theta = %.4f, kappa = %.4f, sigma = %.4f, rho = %.4f' % tuple(params_cal))

/usr/local/lib/python3.9/dist-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Calibrated parameters: v0 = 0.0385, theta = 0.0444, kappa = 0.4759, sigma = 0.2055, rho = -0.4920


In [ ]:
report_calibration(params0, actual_params, params_cal)

,v0,theta,kappa,sigma,rho,Feller
Initial,0.10000,0.05000,0.50000,0.20000,-0.70000,-0.01
Actual,0.04000,0.04000,1.50000,0.30000,-0.50000,-0.03
Calibrated,0.03854,0.04436,0.47591,0.20548,-0.49203,0.00


### XGB

In [ ]:
e_ft_data = pd.read_csv('/content/drive/MyDrive/Practicum/data/eurocall_fourier_100k_final.csv', index_col=0)
X = e_ft_data.drop(['eurocall_ft'], axis=1)
y = e_ft_data['eurocall_ft']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
xgb1 = train_best_reg(xgb.XGBRegressor, X_train, y_train, X_test, y_test,
                      'XGB', **{'colsample_bytree': 0.9, 'eta': 0.05, 'gamma': 0, 'reg_alpha': 0,
                                'reg_lambda': 0, 'min_child_weight': 2, 'n_estimators': 3000, 'max_depth':6})

Current Best XGB Performance:
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, early_stopping_rounds=None,
             enable_categorical=False, eta=0.05, eval_metric=None,
             feature_types=None, gamma=0, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=2, missing=nan,
             monotone_constraints=None, n_estimators=3000, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)
Train Set rmse:  0.00291
XGB rmse:  0.00617 mae: 0.00414
XGB r2:  0.99896


In [ ]:
# xgb1.save_model("model.json")
xgb1 = xgb.XGBRegressor(**{'colsample_bytree': 0.9, 'eta': 0.05, 'gamma': 0, 'reg_alpha': 0,
                           'reg_lambda': 0, 'min_child_weight': 2, 'n_estimators': 3000, 'max_depth':6})
xgb1.load_model("model.json")

In [ ]:
def obj_fun_xgb(params, data):
  v0, theta, kappa, sigma, rho = params
  data_set = np.empty((data.shape[0], 9))
  data_set[:,:4] = data.iloc[:,:4]
  data_set[:,4:] = params
  market_price = data.iloc[:,-1]
  model_price = xgb1.predict(data_set)
  return np.mean((market_price - model_price)**2)

In [ ]:
bounds = [(1e-15, 0.5), (1e-15, 2), (1e-15, 2), (1e-15, 1), (-1, 1)]
nlc = NonlinearConstraint(Feller, -np.inf, 0)
xgb_params_cal = differential_evolution(obj_fun_xgb, bounds=bounds, args=(e_ft,),
                                        maxiter=300, tol=1e-5, constraints=nlc, x0=params0).x
print('Calibrated parameters - XGB: v0 = %.4f, theta = %.4f, kappa = %.4f, sigma = %.4f, rho = %.4f' % tuple(xgb_params_cal))

/usr/local/lib/python3.9/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Calibrated parameters - XGB: v0 = 0.0557, theta = 0.0785, kappa = 0.0270, sigma = 0.0540, rho = 0.6100


In [ ]:
report_calibration(params0, actual_params, xgb_params_cal)

,v0,theta,kappa,sigma,rho,Feller
Initial,0.10000,0.05000,0.50000,0.20000,-0.70,-0.01000
Actual,0.04000,0.04000,1.50000,0.30000,-0.50,-0.03000
Calibrated,0.05571,0.07846,0.02703,0.05395,0.61,-0.00133


In [ ]:
no_of_comb = 20000
e_fourier_test3 = eurocall_fourier_generator(v0, theta, kappa, sigma, rho, no_of_comb, 'eurocall_fourier_test3.csv')

e_ft_data = pd.read_csv('/content/drive/MyDrive/Practicum/data/eurocall_fourier_test3.csv', index_col=0)
X = e_ft_data.drop(['eurocall_ft'], axis=1)
y = e_ft_data['eurocall_ft']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

xgb2 = xgb.XGBRegressor(**{'colsample_bytree': 0.9, 'eta': 0.05, 'gamma': 0, 'reg_alpha': 0,
                           'reg_lambda': 0, 'min_child_weight': 2, 'n_estimators': 3000, 'max_depth':6})

def obj_fun_xgb(params, data):
  v0, theta, kappa, sigma, rho = params
  data_set = np.empty((data.shape[0], 9))
  data_set[:,:4] = data.iloc[:,:4]
  data_set[:,4:] = params
  market_price = data.iloc[:,-1]
  model_price = xgb2.predict(data_set)
  return np.mean((market_price - model_price)**2)

bounds = [(1e-15, 0.5), (1e-15, 2), (1e-15, 2), (1e-15, 1), (-1, 1)]
nlc = NonlinearConstraint(Feller, -np.inf, 0)
xgb_params_cal = differential_evolution(obj_fun_xgb, bounds=bounds, args=(e_fourier_test3,),
                                        maxiter=300, tol=1e-5, constraints=nlc, x0=params0).x
print('Calibrated parameters - XGB: v0 = %.4f, theta = %.4f, kappa = %.4f, sigma = %.4f, rho = %.4f' % tuple(xgb_params_cal))

### LightGBM

In [ ]:
e_ft_data = pd.read_csv('/content/drive/MyDrive/Practicum/data/eurocall_fourier_100k_final.csv', index_col=0)
X = e_ft_data.drop(['eurocall_ft'], axis=1)
y = e_ft_data['eurocall_ft']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
lgb1 = train_best_reg(lgb.LGBMRegressor, X_train, y_train, X_test, y_test,
                      'LGB', **{'colsample_bytree': 0.8, 'learning_rate': 0.05, 'num_leaves': 30, 'reg_alpha': 0.08,
                                'reg_lambda': 0.5, 'subsample': 0.6, 'n_estimators': 6000, 'max_depth':8})

Current Best LGB Performance:
LGBMRegressor(colsample_bytree=0.8, learning_rate=0.05, max_depth=8,
              n_estimators=6000, num_leaves=30, reg_alpha=0.08, reg_lambda=0.5,
              subsample=0.6)
Train Set rmse:  0.00283
LGB rmse:  0.00527 mae: 0.00345
LGB r2:  0.99924


In [ ]:
def obj_fun_lgb(params, data):
  v0, theta, kappa, sigma, rho = params
  data_set = np.empty((data.shape[0], 9))
  data_set[:,:4] = data.iloc[:,:4]
  data_set[:,4:] = params
  market_price = data.iloc[:,-1]
  model_price = lgb1.predict(data_set)
  return np.mean((market_price - model_price)**2)

In [ ]:
bounds = [(1e-15, 0.5), (1e-15, 2), (1e-15, 2), (1e-15, 1), (-1, 1)]
nlc = NonlinearConstraint(Feller, -np.inf, 0)
lgb_params_cal = differential_evolution(obj_fun_lgb, bounds=bounds, args=(e_fourier_test,),
                                        maxiter=200, tol=1e-5, constraints=nlc, x0=params0).x
print('Calibrated parameters - LGB: v0 = %.4f, theta = %.4f, kappa = %.4f, sigma = %.4f, rho = %.4f' % tuple(lgb_params_cal))

Calibrated parameters - LGB: v0 = 0.0377, theta = 0.0812, kappa = 0.0925, sigma = 0.1114, rho = 0.6158


In [ ]:
report_calibration(params0, actual_params, lgb_params_cal)

,v0,theta,kappa,sigma,rho,Feller
Initial,0.1000,0.05000,0.50000,0.20000,-0.70000,-0.01000
Actual,0.0400,0.04000,1.50000,0.30000,-0.50000,-0.03000
Calibrated,0.0377,0.08122,0.09245,0.11137,0.61584,-0.00262


In [ ]:
no_of_comb = 20000
e_fourier_test2 = eurocall_fourier_generator(v0, theta, kappa, sigma, rho, no_of_comb, 'eurocall_fourier_test2.csv')

e_ft_data = pd.read_csv('/content/drive/MyDrive/Practicum/data/eurocall_fourier_test2.csv', index_col=0)
X = e_ft_data.drop(['eurocall_ft'], axis=1)
y = e_ft_data['eurocall_ft']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

lgb2 = train_best_reg(lgb.LGBMRegressor, X_train, y_train, X_test, y_test,
                      'LGB', **{'colsample_bytree': 0.8, 'learning_rate': 0.05, 'num_leaves': 30, 'reg_alpha': 0.08,
                                'reg_lambda': 0.5, 'subsample': 0.6, 'n_estimators': 6000, 'max_depth':8})

def obj_fun_lgb(params, data):
  v0, theta, kappa, sigma, rho = params
  data_set = np.empty((data.shape[0], 9))
  data_set[:,:4] = data.iloc[:,:4]
  data_set[:,4:] = params
  market_price = data.iloc[:,-1]
  model_price = lgb2.predict(data_set)
  return np.mean((market_price - model_price)**2)

bounds = [(1e-15, 0.5), (1e-15, 2), (1e-15, 2), (1e-15, 1), (-1, 1)]
nlc = NonlinearConstraint(Feller, -np.inf, 0)
lgb_params_cal = differential_evolution(obj_fun_lgb, bounds=bounds, args=(e_fourier_test2,),
                                        maxiter=200, tol=1e-5, constraints=nlc, x0=params0).x
print('Calibrated parameters - LGB: v0 = %.4f, theta = %.4f, kappa = %.4f, sigma = %.4f, rho = %.4f' % tuple(lgb_params_cal))

In [ ]:
report_calibration(params0, actual_params, lgb_params_cal)